# CrewAI

Ten notatnik pozwoli Ci zapoznać się z systemami wielo-ageentowymi AI. W trakcie ćwiczenia będziemy wykorzystywać framework [crewAI](https://www.crewai.com/), który omawaliśmy na zajęciach.

Po uzupełnieniu tego notatnika powinieneś/powinnaś wiedzieć:
- jak tworzyć crew i agentów
- jak tworzyć zadania i zlecać ich wykonanie crew
- jak tworzyć systemy sekwencyjne i hierarchiczne

Do wykonania ćwiczeń będą nam potrzebne następujace biblioteki:
```
crewai crewai_tools
```
Dodatkowo, będziemy korzystać z openrouter, by wysyłać zapytania do LLMa. Do tego potrzebujemy API key, a także API base (już podane) i nazwy modelu językowego.
Nazwa modelu powinna być w wpisana następującej formie: "openrounter/{nazwa_modelu}".

In [1]:
!pip install crewai crewai_tools

^C


  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8

In [20]:
import os

API_KEY = ""
API_BASE = "https://openrouter.ai/api/v1"
MODEL_NAME = "openrouter/openai/gpt-4o-mini"


os.environ["OPENAI_API_KEY"] = API_KEY
os.environ["OPENAI_API_BASE"] = API_BASE

Gdy już wybraliśmy model, pora na właściwe zadanie -- tworzenie systemu agentowego.

Dzisiaj stworzysz system do pisania eseju na zadany temat.

## Agenci i narzędzia

Zaczniemy od stworzenia agentów. Będziemy potrzebować:
- researcher, który będzie przeszukiwać sieć, by znaleźć istotne informacje
- planner, który zaplanuje schemat eseju na podstawie treści dostraczonych przez researchera
- writer, który napisze esej na podstawie schematu od planera
- citation manager, który zadba o odpowiednie sformatowanie cytowań
- editor, który sprawdzi esej i przekaże ewentualne uwagi


Agenci są definiowani poprzez rolę (`role`), cel (`goal`), backstory (`backstory`) oraz narzędzia (`tools`). Dodatkowo, możemy im pozwolić na delegowanie zadań lub też nie (`allow_delegation`). W założeniu, nie będziemy pozwalać na delegowanie i będziemy ustawiać wartość tej zmiennej na `False`.



Zacznijmy od tworzenia researchera. Do jego obowiązków będzie należało wyszukiwanie infromacji na zadany przez użytkownika temat. Założmy, że na wyjściu powinien dostarczać listę znalezionych źródeł i krótkie podsumowanie znalezionych informacji.

In [21]:
from crewai import Agent, Task, Crew, LLM, Process

In [22]:
researcher = Agent(
    role="Content Researcher",
    goal=(
        "Provide in-depth, insightful and facutally accurate "
        "analysis and list of appropriate sources "
        "on the topic: {topic}."
    ),
    backstory=(
        "An experienced researcher "
        "able to provide analysis and useful resources on any given topic "
        "when creating a list of resources researcher might need the help of Citation Manager. "
        "Researcher provides only existing resources and makes sure they are relevant. "
        "Researcher's work is the basis for the Content Planner and Content Writer to "
        "write the outline and the essay."
    ),
    verbose=True,
    allow_delegation=False,
    max_iterations=2,
    llm=LLM(
        model=MODEL_NAME,
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url=API_BASE,
        temperature=0.7,
    ),
)

LLM value is already an LLM object


Następnie tworzymy plannerai writera.

Planner ma za zadanie zaplanować outline eseju.
Writer na podstawie takiego outline'u ma napisać esej.

In [23]:
planner = Agent(
    role="Content Planner",
    goal="Plan insightful and factually accurate content on {topic}",
    backstory=(
        "An agent that is working on planning an essay "
        "on the topic: {topic}."
        "Content Planner collects information with the help of Content Researcher. "
        "Based on the information, Content Planner writes the outline of the essay. "
        "Content Planner also includes the Researcher's list of sources and relevant information. "
        "Planner's work is the basis for "
        "the Content Writer to write an article on this topic."
    ),
    verbose=True,
    allow_delegation=False,
    max_iterations=2,
    llm=LLM(
        model=MODEL_NAME,
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url=API_BASE,
        temperature=0.7,
    ),
)

LLM value is already an LLM object


In [24]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate essay on the topic: {topic}",
    backstory=(
        "A agent working on a writing "
        "an essay on the topic: {topic}. "
        "Content Writer bases their writing on the work of "
        "the Content Planner, who provides an outline "
        "and relevant context about the topic. "
        "Content Writer follows the main objectives and "
        "direction of the outline, "
        "provided by the Content Planner. "
        "Content Writer also provides objective and impartial insights "
        "and backs them up with information "
        "provided by the Content Planner. "
        "Content Writer uses evidence and reasoning. "
        "Content Writer makes sure to back-up each argument with a proper source, "
        "provided by Content Planner."
    ),
    verbose=True,
    allow_delegation=False,
    max_iterations=2,
    llm=LLM(
        model=MODEL_NAME,
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url=API_BASE,
        temperature=0.7,
    ),
)

LLM value is already an LLM object


Tworzymy kolejnego agenta: editora. Editor będzie miał za zadanie poprawiać esej dostarczony przez writera.

In [25]:
editor = Agent(
    role="Editor",
    goal=(
        "Edit a given essay on the topic: {topic} to align with "
        "the writing style of the organization. "
    ),
    backstory=(
        "An agent that is an editor who receives an essay "
        "from the Content Writer. "
        "Editor's goal is to review the essay "
        "to ensure that it follows best practices:"
        "analyzes the topic, defines arguments, uses evidence and reasoning and is written coherently."
        "Editor makes sure that the essay provides balanced viewpoints "
        "and also avoids major controversial topics "
        "or opinions when possible."
        "Editor also checks whether all provided sources actually exist and are used properly."
        "Editor makes sure that the cited sources are cited in the MLA style of citations."
    ),
    allow_delegation=False,
    verbose=True,
    max_iterations=2,
    llm=LLM(
        model=MODEL_NAME,
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url=API_BASE,
        temperature=0.7,
    ),
)

LLM value is already an LLM object


Ostatnim członkiem naszej załogi będzie citation manager.

**Zadanie 1**

**Stwórz agenta o roli Citation Manager. Jego celem jest sformatowanie cytowań i źródeł w stylu MLA. Opisz odpoiwednio rolę, cel i backstory dla tego agenta. Następnie ustaw zmienne: `allow_delegation=False`, `verbose=True`, `max_iterations=2`. Przypisz agentowi LLMa opartego na gpt-4o-mini (tak jak w przypadku poprzednio tworzonych agentów).**

In [26]:
citation_manager = Agent(
    role="Citation Manager",
    goal="Format and verify all citations and sources in MLA style for the essay on {topic}",
    backstory=(
        "An expert citation specialist who ensures all academic references "
        "are properly formatted according to MLA guidelines. "
        "Works closely with the Content Writer and Editor to verify "
        "that all in-text citations and the works cited list "
        "follow proper MLA formatting rules. "
        "Helps maintain academic integrity by ensuring "
        "all sources are properly attributed and formatted consistently. "
        "Has extensive experience in academic writing and "
        "is particularly skilled in MLA citation style."
    ),
    verbose=True,
    allow_delegation=False,
    max_iterations=2,
    llm=LLM(
        model=MODEL_NAME,
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url=API_BASE,
        temperature=0.7,
    ),
)

LLM value is already an LLM object


## Zadania

Teraz tworzymy zadanie dla każdego ze stworzonych agentów.
Każde zadanie zawiera opis, oczekiwany output oraz opcjonalnie agenta, który je wykonuje.

Kolejno definujemy zadania "research", "plan", "write", "edit".

In [27]:
research = Task(
    description=(
        "1. Conduct comprehensive research on the topic: {topic}.\n"
        "2. Prioritize identifying the latest trends, impactful arguments, and noteworthy news related to {topic}.\n"
        "3. Curate a list of relevant and reliable resources to support the essay's arguments.\n"
        "4. Organize findings into key categories to assist the Content Planner in developing a structured outline.\n"
        "5. Provide a brief summary for each resource explaining its relevance to the topic.\n"
        "6. Make sure to only provide factually accurate information.\n"
    ),
    expected_output=(
        "A well-organized research document containing:\n"
        "- A categorized summary of key findings.\n"
        "- A list of reliable and relevant resources with explanations.\n"
        "- Insights into recent trends, impactful arguments, and noteworthy news.\n"
    ),
    agent=researcher,
)

In [28]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, impactful arguments, "
            "and noteworthy news on {topic}.\n"
        "2. Develop a detailed essay content outline including "
            "an introduction, key points, and a call to action.\n"
        "3. Include SEO keywords and relevant data or sources obtained from the Content Researcher.\n"
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and properly formatted resources.",
    agent=planner,
)

In [29]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "essay on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		    "3. Sections/Subtitles are properly named "
            "in an academical manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Make sure that each presented argument is supported by a proper citation.\n"
        "6. Proofread for grammatical errors and "
            "alignment with the academical style.\n"
        "7. Include a list of used sources used.\n"
    ),
    expected_output="A well-written essay "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs. At the end, there should be a list of sources used. "
        "The answer should include only the essay and list of sources, without any additional information.",
    agent=writer,
)

In [30]:
edit = Task(
    description=(
        "1. Review the essay on the topic: {topic} for coherence, readability, and alignment with the organization's writing style.\n"
        "2. Ensure the essay:\n"
        "   - Analyzes the topic thoroughly and defines arguments clearly.\n"
        "   - Uses evidence and reasoning to support each argument.\n"
        "   - Avoids major controversial topics or opinions unless explicitly required.\n"
        "   - Provides balanced viewpoints where applicable.\n"
        "3. Verify the existence and relevance of all cited sources.\n"
        "4. Check whether all sources are cited in proper MLA format.\n"
        "5. Proofread the essay for grammatical accuracy, stylistic consistency, and adherence to best practices in essay writing.\n"
        "6. Ensure the final content is engaging, error-free, and ready for publication.\n"
    ),
    expected_output=(
        "A polished and edited essay with:\n"
        "- Improved coherence and alignment with the organization's writing style.\n"
        "- Verified and properly formatted MLA citations.\n"
        "- Balanced arguments supported by evidence and reasoning.\n"
        "- Grammatical and stylistic accuracy.\n"
        "The output should only include the essay and list of resource, without any additional information."
    ),
    agent=editor,
)

**Zadanie 2**

**Napisz zadanie manage_citations. Zadanie powinno polegać na: po pierwsze, zweryfikowaniu źródeł z eseju, sformatowaniu ich w odpowiedni sposób stosując format MLA, dobrym zorganizowaniu sformatowanych źródeł. Efektem zadania powinna być własnie taka odpowiednio sformatowana lista i poprawnie sformatowane cytowania w tekście. Przypisz do zadania agenta `citation_manager`.**

In [31]:
manage_citations = Task(
    description=(
        "1. Review all sources and citations used in the essay.\n"
        "2. Verify the accuracy and existence of each source.\n"
        "3. Format all in-text citations according to MLA style guidelines.\n"
        "4. Create a properly formatted Works Cited list in MLA style, including:\n"
        "   - Correct ordering (alphabetical by author's last name)\n"
        "   - Proper formatting for different types of sources\n"
        "   - Required elements for each citation (author, title, publisher, date, etc.)\n"
        "5. Ensure consistency between in-text citations and Works Cited entries.\n"
        "6. Organize the formatted sources in a clear and readable manner.\n"
    ),
    expected_output=(
        "A document containing:\n"
        "- Properly formatted in-text citations\n"
        "- A complete Works Cited list in MLA format\n"
        "- All sources verified and correctly formatted\n"
        "The output should be ready to be integrated into the final essay."
    ),
    agent=citation_manager
)

## Tworzenie crew

Mając już określonych agentów oraz zadania musimy skonstrkuować crew.

**Zadanie 3**

**Stwórz crew do pisania eseju uzupełniając parametry `agents` i `tasks`. Te parametry przyjmują listy odpowiednio agentów lub tasków. Przy taskach, wypisz zadania w odpowiedniej kolejności, tak aby proces przebiegł prawidłowo. Dodatkowo ustaw zmiennę `verbose=True`**


In [32]:
crew = Crew(
    agents=[researcher, planner, writer, citation_manager, editor],
    tasks=[research, plan, write, manage_citations, edit],
    verbose=True,
    process=Process.sequential
)

## Pisanie eseju

Teraz zobaczymy jak nasz system radzi sobie z zadaniem.

**Zadanie 4**

**Wybierz dowolny temat eseju.**

In [35]:
essay_topic = "Usage of actor model in concurrent computing"

result = crew.kickoff(inputs={"topic": essay_topic})

# Agent: Content Researcher
## Task: 1. Conduct comprehensive research on the topic: Usage of actor model in concurrent computing.
2. Prioritize identifying the latest trends, impactful arguments, and noteworthy news related to Usage of actor model in concurrent computing.
3. Curate a list of relevant and reliable resources to support the essay's arguments.
4. Organize findings into key categories to assist the Content Planner in developing a structured outline.
5. Provide a brief summary for each resource explaining its relevance to the topic.
6. Make sure to only provide factually accurate information.



# Agent: Content Researcher
## Final Answer: 
### Research Document: Usage of Actor Model in Concurrent Computing

#### I. Introduction to the Actor Model
The actor model is an abstraction used in concurrent computing that treats "actors" as the fundamental units of computation. Each actor can send and receive messages, create new actors, and maintain its own state. This model simpl

Zobaczmy teraz jak wygląda nasz esej.

In [36]:
from IPython.display import Markdown
Markdown(str(result))

# Usage of Actor Model in Concurrent Computing

## I. Introduction to the Actor Model

The actor model is a conceptual framework for modeling concurrent computation, wherein "actors" serve as the fundamental units of computation. Each actor encapsulates state and behavior, communicating with other actors through asynchronous message passing. This abstraction simplifies the complexity often associated with concurrent system development, enabling developers to create scalable and fault-tolerant systems without the burdens traditionally imposed by shared mutable state. Unlike traditional programming models that rely on locks and shared memory, the actor model promotes a more natural and intuitive way to handle concurrency, aligning well with modern computing paradigms.

The relevance of the actor model has surged as the demand for distributed systems and cloud-native applications has increased. By decoupling the components of a system, the actor model not only enhances scalability but also reduces the risk of contention and deadlocks, which are common pitfalls in concurrent programming. This essay explores the key findings related to the actor model's application in concurrent computing, emphasizing its advantages in scalability, fault tolerance, and real-world use cases.

## II. Key Findings

### 1. Concurrency Simplification

One of the primary advantages of the actor model is its ability to simplify concurrency. By encapsulating state within actors, the model effectively eliminates the risks associated with shared mutable state, which can lead to race conditions and other synchronization issues. Instead of using locks to manage access to shared resources, actors communicate through message passing, which inherently avoids many of the complexities associated with traditional synchronization mechanisms. This approach not only improves code readability but also enhances overall system reliability (Agha).

Message passing allows for a more modular design, where actors can operate independently and asynchronously. This decoupling of components aligns well with the principles of functional programming, which emphasizes immutability and statelessness. As a result, developers can more easily reason about their code, leading to fewer bugs and more maintainable systems (Armstrong).

### 2. Scalability and Distribution

The actor model is particularly well-suited for cloud-based applications, where scalability is a critical requirement. By distributing actors across multiple nodes, systems can efficiently utilize resources while maintaining high levels of performance. This scalability is further enhanced by the model's compatibility with microservices architecture, which promotes the development of loosely coupled services that can be deployed and scaled independently (Newman).

In a microservices environment, actors can serve as independent services that communicate with one another through messages. This not only improves resource utilization but also allows for dynamic scaling based on load. As demand fluctuates, new actor instances can be spawned or terminated, ensuring optimal performance without the overhead of managing shared state (Van Roy).

### 3. Fault Tolerance

Fault tolerance is another crucial aspect of the actor model, where the "let it crash" philosophy plays a significant role. In actor-based systems, each actor can fail independently without affecting the entire system. Supervisory strategies, such as supervisors that monitor and restart failed actors, enhance resilience and ensure that the system can recover gracefully from errors (Kuhn et al.).

This approach to fault tolerance is particularly beneficial in high-availability environments, such as telecommunications and finance, where system reliability is paramount. By isolating failures and employing supervision, actor-based systems can achieve a level of robustness that is challenging to attain with traditional models that rely on shared state (Armstrong).

### 4. Real-World Applications

The actor model has found significant application in various industries, particularly in those requiring high concurrency and reliability. In telecommunications, for example, actor-based systems manage millions of concurrent connections, ensuring that each actor can handle requests independently without bottlenecking the system. Similarly, in the gaming industry, where real-time interactions are crucial, the actor model facilitates the management of numerous game entities, enabling seamless multiplayer experiences (Agha).

In finance, actor-based frameworks are employed to build systems that can process transactions concurrently while maintaining strict consistency and fault tolerance. The ability of the actor model to handle high concurrency without compromising performance has made it a preferred choice for developing reliable applications in these demanding domains (Newman).

## III. Trends and Impactful Arguments

### Increased Adoption of Functional Programming

The alignment of the actor model with functional programming languages, such as Scala and Elixir, has led to increased adoption among developers. Functional programming emphasizes immutability and statelessness, principles that resonate with the actor model's design. As developers seek to leverage the benefits of both paradigms, the actor model becomes an attractive option for building robust, concurrent applications (Van Roy).

### Integration with Cloud Computing

The rise of cloud-native applications has further propelled the adoption of the actor model. Actor frameworks, such as Akka and Microsoft Orleans, provide powerful tools for developers to build scalable and resilient applications in cloud environments. These frameworks streamline the process of creating actor-based systems, allowing developers to focus on business logic rather than the complexities of concurrency and distribution (Kuhn et al.).

### Emergence of Actor Frameworks

The growing popularity of actor frameworks, such as Akka, Microsoft Orleans, and Erlang/OTP, highlights the increasing recognition of the actor model's advantages in concurrent computing. These frameworks provide developers with the necessary tools to implement actor-based systems efficiently, promoting best practices and facilitating the development of distributed applications (Agha).

## IV. Conclusion

In summary, the actor model offers significant advantages in the realm of concurrent computing. Its ability to simplify concurrency, enhance scalability, and provide robust fault tolerance makes it an essential framework for modern software development. As the demand for distributed systems and cloud-native architectures continues to rise, the actor model's relevance will only grow.

Developers are encouraged to explore the provided resources to gain a deeper understanding of the actor model's applications and implications. By experimenting with actor-based programming in their projects, they can experience firsthand the benefits of this powerful paradigm and contribute to the ongoing evolution of concurrent computing.

## Works Cited

Agha, Gul. *Actors: A Model of Concurrent Computation in Distributed Systems*. MIT Press, 1986.

Armstrong, Joe. *Concurrent Programming in Erlang*. Prentice Hall, 2007.

Kuhn, R., et al. *Akka: Concurrency in Scala*. Artima Press, 2010.

Newman, Sam. *Building Microservices: Designing Fine-Grained Systems*. O'Reilly Media, 2015.

Van Roy, Peter. *Actor-Based Programming: The Actor Model in Practice*. Cambridge University Press, 2004.

## Procesy hierarchiczne

Póki co korzystaliśmy tylko z procesu sekwencyjnego i dokładnie musieliśmy określić każde z zadań. Teraz spróbujemy użyć dodatkowego agenta -- managera, który będzie zarządzał pozostałymi agentami, w celu napisania eseju.

Nie będziemy tutaj musieli tak dokładnie rozpisywać zadań. Określimy tylko ogólne zadanie "pisanie eseju", a resztą zajmie się manager.

**Możesz dostosować opis managera, żeby był bardziej dopasowany do konkretnego zadania pisania eseju.**

In [37]:
MODEL_NAME_MANAGER = 'openrouter/openai/gpt-3.5-turbo'

In [38]:
manager = Agent(
    role="Project Manager",
    goal="Efficiently manage the crew and ensure high-quality task completion by delegating the tasks. ",
    backstory=(
        "You are an experienced project manager, "
        "skilled in overseeing complex projects and "
        "guiding teams to success. "
        "Your role is to coordinate the efforts of all of the crew members, "
        "ensuring that each task is completed to the highest standard."
        "You should use the expertise of the other crew members to correctly perform smaller tasks.  "
        "You are very busy and should not perform any task on your own, just focus on proper delegation."
    ),
    allow_delegation=True,
    verbose=True,
    max_iterations=15,
    llm=LLM(
        model=MODEL_NAME_MANAGER,
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url=API_BASE,
        temperature=0.7,
    )
)

LLM value is already an LLM object


In [39]:
task = Task(
    description=(
        "Write an essay on the given topic: {topic}. "
        "The essay should adhere to the rules of writing a good essay. "
        "The essay should be well-writted and have a clear structure. "
        "Each section of the essay should have 2 to 3 paragraphs. "
        "Each argument presented in the essay should be supported by a proper and relevant citation. "
        "The citations should be fatually accurate and the sources should exist. "
        "At the end of the essay, there should be a list of used sources in the MLA citation style "
        "with the help of Citation Manager. "
        "The essay should be verified by the Editor before final submission. "
    ),
    expected_output=(
        "A well-written essay "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs. In the end, there should be a list of sources used. "
        "The output should only include the essay and list of resource, without any additional information. "
    ),
)

Jak widać, w taki sposób możemy zaoszczędzić dużo wysiłku na pisanie samego procesu. Udało nam się streścić główny zamysł w jednym zadaniu i liczymy na to, że resztą zajmie się manager.

Pora na zdefiniowanie crew. Tutaj właśnie użyjemy procesu hierarchicznego.

**Zadanie 5**

**Stwórz crew do pisania eseju, wykorzysując proces hierarchiczny. Pamiętaj, że agent `manager` nie powinien być wpisany w listę agentów -- poszukaj w [dokumentacji crewAI](https://docs.crewai.com/concepts/crews), w jaki sposób go użyć. Crew powinna otrzymać tylko jedengo taska (tego zdefiniowanego powyżej).**

In [41]:
crew = Crew(
    agents=[researcher, planner, writer, citation_manager, editor],
    tasks=[task],
    verbose=True,
    process=Process.hierarchical,
    manager_agent=manager
)

Użyjemy tego samego tematu, co poprzednio.

In [42]:
result = crew.kickoff(inputs={"topic": essay_topic})

# Agent: Project Manager
## Task: Write an essay on the given topic: Usage of actor model in concurrent computing. The essay should adhere to the rules of writing a good essay. The essay should be well-writted and have a clear structure. Each section of the essay should have 2 to 3 paragraphs. Each argument presented in the essay should be supported by a proper and relevant citation. The citations should be fatually accurate and the sources should exist. At the end of the essay, there should be a list of used sources in the MLA citation style with the help of Citation Manager. The essay should be verified by the Editor before final submission. 
# Agent: Content Researcher
## Task: Research relevant information on the actor model in concurrent computing


# Agent: Content Researcher
## Final Answer: 
The actor model is a mathematical model for concurrent computation that treats "actors" as the fundamental units of computation. This model provides a way to design systems that can handle 

Niewykluczone, że w pewnym momencie zobaczyłeś/zobaczyłaś błąd, informujący o problemie z delegacją zadania lub nasz manager nie wykorzystał pełnego potencjału crew. Należy pamiętać, że mimo wszystko mamy do czynienia z LLMami -- czasami taki model nie dostosuje się do formatu, którego oczekuje crewAI, co potencjalnie doprowadzi do błędu. Dodatkowo, LLM może pójść na skróty lub zgubić się ze własnym rozumowaniu, przez co nie oddeleguje zadań w pełni poprawnie.

In [43]:
from IPython.display import Markdown
Markdown(str(result))

**The Usage of the Actor Model in Concurrent Computing**

In the landscape of concurrent computing, the need for robust and efficient models to manage the complexity of concurrent processes has never been more critical. One such model that has gained significant traction in recent years is the Actor Model. This essay explores the principles of the Actor Model, its advantages in concurrent computing, its real-world applications, and the challenges it faces.

**Understanding the Actor Model**

The Actor Model, introduced by Carl Hewitt in the 1970s, represents a conceptual framework where "actors" serve as the fundamental units of computation. Each actor is an independent entity that encapsulates its state and behavior, and communicates with other actors through asynchronous message passing. This decoupled communication allows for a high degree of concurrency, as actors do not share state but instead operate on their own internal data. This model contrasts sharply with traditional thread-based concurrency, where shared states can lead to issues such as race conditions and deadlocks.

**Advantages of the Actor Model**

One of the primary advantages of the Actor Model is its ability to simplify the design of concurrent systems. Since actors communicate through messages rather than shared memory, developers can create systems that are inherently more modular and easier to reason about. This modularity allows for better scalability, as actors can be distributed across multiple nodes in a network without requiring intricate synchronization mechanisms. 

Moreover, the Actor Model provides built-in fault tolerance. Each actor can process messages independently, so if one actor fails, it does not necessarily compromise the entire system. Instead, supervisory actors can monitor and restart failed actors, ensuring that the system remains operational. This resilience is crucial for applications that require high availability, such as financial systems or real-time data processing.

**Real-World Applications**

The Actor Model has found practical applications across various domains. One of the most notable implementations is in the Akka framework for building concurrent, distributed, and resilient message-driven applications on the JVM (Java Virtual Machine). Akka enables developers to build highly scalable systems by leveraging the Actor Model principles, allowing for seamless communication between actors across different servers.

Another significant example is the Erlang programming language, designed for building scalable and fault-tolerant applications. Erlang’s concurrency model is based on actors, enabling developers to create systems that can handle thousands of simultaneous users without performance degradation. Applications such as WhatsApp and the telecom systems that support it have successfully utilized Erlang’s Actor Model for their scalable architectures.

**Challenges of the Actor Model**

Despite its advantages, the Actor Model is not without challenges. One of the primary hurdles is the learning curve associated with adopting a new concurrency paradigm. Developers accustomed to traditional threading models may find it difficult to adapt to the asynchronous nature of message passing and the absence of shared state.

Additionally, debugging systems built on the Actor Model can be complex. Since actors operate independently and communicate asynchronously, tracing the flow of messages and identifying the source of faults can be challenging. Tools and methodologies for debugging actor-based systems are still evolving, making this a critical area for future research and development.

**Conclusion**

The Actor Model presents a powerful paradigm for managing concurrency in computing. Its principles of modularity, resilience, and scalability make it particularly well-suited for today’s distributed systems. As applications continue to demand more from concurrent computing, understanding and implementing the Actor Model can empower developers to create robust, efficient, and fault-tolerant systems. While challenges remain in terms of adoption and debugging, the benefits offered by the Actor Model position it as a vital component in the future of concurrent computing. 

By leveraging the Actor Model, developers can navigate the complexities of concurrent systems more effectively, ensuring that their applications not only meet the demands of today but are also prepared for the challenges of tomorrow.

**Zadanie 6**

**Którym sposobem otrzymaliśmy lepszy esej? Czy eseje mocno różnią się od siebie? Możesz też porównać esej wygenerowany przy pomocy crew z esejem, który otrzymasz używając ChataGPT (najlepiej z gpt-4o-mini).**

1. **Structure and Organization**
- Essay 1 (Sequential Process) is more academically structured with clear sections, subsections, and a formal Works Cited section in MLA format
- Essay 2 (Hierarchical Process) has a simpler structure with bold headers and fewer subsections
- Essay 1 appears more polished and better organized

2. **Content Depth**
- Essay 1 provides more detailed coverage with specific examples and trends
- Essay 1 includes more technical details and industry-specific applications
- Essay 2 offers a more general overview of the topic

3. **Citations and Sources**
- Essay 1 includes proper in-text citations and a formal Works Cited section
- Essay 2 lacks formal citations, making it less academically rigorous
- Essay 1 demonstrates better use of the Citation Manager agent

**Conclusion:**
The sequential process (Essay 1) produced a better result because:
1. It followed a more structured workflow where each agent had a specific role
2. The Citation Manager and Editor had clear, dedicated steps in the process
3. The sequential nature ensured that each aspect (research, planning, writing, citations, editing) was properly addressed

The hierarchical process (Essay 2), while still producing a decent essay, didn't fully utilize the specialized agents' capabilities, particularly regarding citations and academic formatting. This might be because the manager agent didn't effectively delegate all aspects of the task.

For academic writing, the sequential process appears to be more reliable and produces more polished results.